### Experiment to remove all the patients that passed away in the hospital

In [11]:
import re
import numpy as np
import pandas as pd
import nltk
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score, RandomizedSearchCV
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from sklearn.metrics import mean_squared_error
import json  


# Download wordnet
nltk.download('wordnet')

# Optional: You might also want to download the punkt tokenizer and stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/asheshlalshrestha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/asheshlalshrestha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/asheshlalshrestha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
df = pd.read_csv("../Datasets/new_with_expire_column.csv")
df = df.drop("Unnamed: 0",axis=1)
df = df[df['did_the_patient_expire_in_hospital']=='No']

In [13]:
df

,id,did_the_patient_expire_in_hospital,hospital_length_of_stay,age,sex,height,weight,smoking_history,previous_er_visit_within_14_days,admission_disposition,...,Respiractin,Immunity Advance,Vesicare,Zaxine,Quinine Sulfate,Desvenlafaxine,Glucosamine,Turmeric,Cogentin Tab,Elavil
1,2,No,5,61,0,154.900000,73.600000,0.0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,No,7,58,0,166.911029,80.265922,0.0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,No,32,44,1,177.000000,112.000000,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,7,No,4,50,1,188.000000,91.100000,0.0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,8,No,10,72,1,171.000000,110.000000,1.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,514,No,4,68,0,164.000000,77.100000,0.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
501,515,No,2,56,0,165.000000,72.600000,0.0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
502,516,No,13,74,0,166.911029,55.000000,0.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
504,519,No,9,74,1,166.911029,81.700000,0.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


### Now matching with the comorbidities

In [14]:
# Load data
xls = pd.ExcelFile('/Users/asheshlalshrestha/Desktop/UB/Research/CarePathways/Models/Datasets/Canada_Hosp1_COVID_InpatientData.xlsx')
comorbidities = pd.read_excel(xls, 'Data-at-admission')
comorbidities = comorbidities[['id','comorbidities']]
comorbidities
# df = df.merge(df1, on='id')
# df

,id,comorbidities
0,1,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]"""
1,2,"""[\""Hypertension\"",\""Other\""]"""
2,3,"""[\""Hypertension\""]"""
3,4,"""[\""Hypertension\"",\""Other\""]"""
4,5,"""[\""Chronic cardiac disease (not hypertension)..."
...,...,...
503,516,"""[\""Other\""]"""
504,517,"""[\""Hypertension\"",\""Chronic renal\"",\""Diabete..."
505,519,"""[\""Hypertension\"",\""Other\""]"""
506,521,"""[]"""


### Applying Word2Vec

In [15]:
def clean_string(s):
    return [part.strip('"') for part in s.replace('\\', '').replace('"', '').strip('[]').split(',')]

# Apply the function to clean and generate sentence vectors
comorbidities['comorbidities'] = comorbidities['comorbidities'].apply(clean_string)

# Train Word2Vec model
model = Word2Vec(sentences=comorbidities['comorbidities'], vector_size=50, window=5, min_count=1, sg=1)

# Function to convert comorbidities to vectors
def comorbidities_to_vector(comorbidity_list):
    return [model.wv[word] for word in comorbidity_list if word in model.wv.key_to_index]

# Apply the function to create a new column with vectors
comorbidities['vectorized_comorbidities'] = comorbidities['comorbidities'].apply(comorbidities_to_vector)

In [16]:
comorbidities

,id,comorbidities,vectorized_comorbidities
0,1,"[Hypertension, Diabetes, Other]","[[-0.018746352, 0.009513315, -0.007519254, 0.0..."
1,2,"[Hypertension, Other]","[[-0.018746352, 0.009513315, -0.007519254, 0.0..."
2,3,[Hypertension],"[[-0.018746352, 0.009513315, -0.007519254, 0.0..."
3,4,"[Hypertension, Other]","[[-0.018746352, 0.009513315, -0.007519254, 0.0..."
4,5,"[Chronic cardiac disease (not hypertension), H...","[[0.013756036, -0.018520297, 4.1683903e-05, 0...."
...,...,...,...
503,516,[Other],"[[-0.0032618782, 0.0008874875, 0.010716496, 0...."
504,517,"[Hypertension, Chronic renal, Diabetes, Other]","[[-0.018746352, 0.009513315, -0.007519254, 0.0..."
505,519,"[Hypertension, Other]","[[-0.018746352, 0.009513315, -0.007519254, 0.0..."
506,521,[],"[[0.002874055, -0.0052920175, -0.014147566, -0..."


### The hyper parameter tuning for word to vector model needs some medical intuition as we need to find the similar words for our reference word i.e. Hypertension
Thus we will choose Vector Size 50 and Window 5 (SG=1):

Neighbors: ['Pregnancy', 'Asthma', 'Other', 'Chronic neurological disorder', 'Chronic renal']

as our best model

In [17]:
import itertools

# Define hyperparameter ranges for grid search
vector_sizes = [50, 100]  # You can expand this range
windows = [2, 5]           # You can expand this range
min_counts = [1, 5]        # You can expand this range
sgs = [0, 1]               # CBOW and Skip-gram

# List to store results
results = []

# Perform grid search
for vector_size, window, min_count, sg in itertools.product(vector_sizes, windows, min_counts, sgs):
    # Train Word2Vec model
    model = Word2Vec(sentences=comorbidities['comorbidities'], 
                     vector_size=vector_size, 
                     window=window, 
                     min_count=min_count, 
                     sg=sg)

    # Evaluate the model with a sample word
    sample_word = "Hypertension"  # Example word for evaluation
    try:
        if sample_word in model.wv.key_to_index:
            neighbors = model.wv.most_similar(sample_word, topn=5)
            results.append((vector_size, window, min_count, sg, neighbors))
        else:
            results.append((vector_size, window, min_count, sg, "Word not in vocabulary"))
    except Exception as e:
        results.append((vector_size, window, min_count, sg, str(e)))

# Display the results of the grid search
print("Hyperparameter Combinations and Nearest Neighbors:")
for result in results:
    print(f"Vector Size: {result[0]}, Window: {result[1]}, Min Count: {result[2]}, SG: {result[3]}, Neighbors: {result[4]}")

Hyperparameter Combinations and Nearest Neighbors:
Vector Size: 50, Window: 2, Min Count: 1, SG: 0, Neighbors: [('Pregnancy', 0.22752653062343597), ('Asthma', 0.16822075843811035), ('Chronic neurological disorder', 0.16588544845581055), ('Chronic renal', 0.12444842606782913), ('Other', 0.1211235448718071)]
Vector Size: 50, Window: 2, Min Count: 1, SG: 1, Neighbors: [('Pregnancy', 0.22945445775985718), ('Asthma', 0.19182366132736206), ('Chronic neurological disorder', 0.17280414700508118), ('Other', 0.17043617367744446), ('Chronic renal', 0.1539248526096344)]
Vector Size: 50, Window: 2, Min Count: 5, SG: 0, Neighbors: [('Asthma', 0.1487571895122528), ('Other', 0.10766202211380005), ('History of cancer [now in remission]', 0.10266797989606857), ('Chronic renal', 0.08812781423330307), ('Diabetes', 0.08045753836631775)]
Vector Size: 50, Window: 2, Min Count: 5, SG: 1, Neighbors: [('Asthma', 0.1514696329832077), ('Other', 0.1173236072063446), ('History of cancer [now in remission]', 0.11023

In [19]:
df = df.merge(comorbidities, on='id')
df.to_csv('new_with_word2vec_comorbidities.csv')

In [6]:
import re
import numpy as np
import pandas as pd
import nltk
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import json

# Download necessary NLTK resources
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

# Load data
df = pd.read_csv("../Datasets/new_with_expire_column.csv")
df = df.drop("Unnamed: 0", axis=1)
df = df[df['did_the_patient_expire_in_hospital'] == 'No']

xls = pd.ExcelFile('/Users/asheshlalshrestha/Desktop/UB/Research/CarePathways/Models/Datasets/Canada_Hosp1_COVID_InpatientData.xlsx')
comorbidities = pd.read_excel(xls, 'Data-at-admission')
comorbidities = comorbidities[['id', 'comorbidities']]

# Clean comorbidities strings
def clean_string(s):
    return [part.strip('"') for part in s.replace('\\', '').replace('"', '').strip('[]').split(',')]

comorbidities['comorbidities'] = comorbidities['comorbidities'].apply(clean_string)

# Train Word2Vec model (CBOW with embedding dimension 100)
model = Word2Vec(
    sentences=comorbidities['comorbidities'],
    vector_size=100,   # Embedding dimension
    window=5,
    min_count=1,
    sg=0               # CBOW model
)

# Function to convert comorbidities to vectors and aggregate them
def comorbidities_to_vector(comorbidity_list):
    vectors = [model.wv[word] for word in comorbidity_list if word in model.wv.key_to_index]
    if not vectors:
        return np.zeros(model.vector_size)  # Return zero vector if no words found
    return np.mean(vectors, axis=0)  # Aggregate by mean

# Apply the function to create a new column with aggregated vectors
comorbidities['vectorized_comorbidities'] = comorbidities['comorbidities'].apply(comorbidities_to_vector)

# Use "real names" for columns. Here, they're placeholders, replace with domain-specific names if available.
real_names = [f'feature_{i}' for i in range(model.vector_size)]  # Replace with meaningful names

# Split the vectors into separate columns
vectorized_df = pd.DataFrame(comorbidities['vectorized_comorbidities'].tolist(), columns=real_names)

# Concatenate the original DataFrame with the vectorized columns
comorbidities = pd.concat([comorbidities, vectorized_df], axis=1)

# Drop the aggregated vector column if not needed
comorbidities = comorbidities.drop(columns=['vectorized_comorbidities'])

# Save the updated DataFrame
comorbidities.to_csv('comorbidities_with_vectors.csv', index=False)

comorbidities

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/asheshlalshrestha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/asheshlalshrestha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/asheshlalshrestha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,comorbidities,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99
0,1,"[Hypertension, Diabetes, Other]",-0.003413,0.003305,0.001668,0.005053,0.002423,-0.003192,0.001913,0.007787,...,-0.000256,0.001633,0.005509,-0.004061,0.006309,0.004526,0.001771,-0.003166,0.000249,0.001823
1,2,"[Hypertension, Other]",-0.005018,0.002996,0.005732,0.008072,-0.000451,-0.007979,0.004482,0.009612,...,0.001741,-0.000491,0.003457,-0.003979,0.004978,0.004051,-0.000673,-0.004983,-0.003847,0.006019
2,3,[Hypertension],-0.009010,0.004666,0.005772,0.006445,0.007928,-0.007510,0.001802,0.008155,...,0.001509,-0.001390,0.002812,-0.008017,-0.001268,0.002852,0.006459,-0.002609,-0.009086,0.005165
3,4,"[Hypertension, Other]",-0.005018,0.002996,0.005732,0.008072,-0.000451,-0.007979,0.004482,0.009612,...,0.001741,-0.000491,0.003457,-0.003979,0.004978,0.004051,-0.000673,-0.004983,-0.003847,0.006019
4,5,"[Chronic cardiac disease (not hypertension), H...",-0.004704,0.005036,0.001325,0.003440,0.003076,-0.003704,0.002261,0.008015,...,-0.001961,0.001006,0.004053,-0.003707,0.007453,0.003295,0.003002,-0.004253,-0.000350,0.000117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,516,[Other],-0.001025,0.001326,0.005691,0.009699,-0.008830,-0.008447,0.007162,0.011068,...,0.001973,0.000408,0.004101,0.000060,0.011223,0.005250,-0.007805,-0.007356,0.001392,0.006872
504,517,"[Hypertension, Chronic renal, Diabetes, Other]",-0.000579,0.001490,0.001051,0.004121,0.001822,-0.002271,0.003043,0.006073,...,-0.000819,0.001356,0.003311,-0.003164,0.004724,0.003621,0.003507,-0.003851,-0.000190,0.002822
505,519,"[Hypertension, Other]",-0.005018,0.002996,0.005732,0.008072,-0.000451,-0.007979,0.004482,0.009612,...,0.001741,-0.000491,0.003457,-0.003979,0.004978,0.004051,-0.000673,-0.004983,-0.003847,0.006019
506,521,[],-0.008727,0.002130,-0.000874,-0.009319,-0.009428,-0.001411,0.004432,0.003704,...,0.009071,0.008938,-0.008208,-0.003012,0.009887,0.005104,-0.001588,-0.008692,0.002962,-0.006676


In [8]:
df = df.merge(comorbidities, on='id')
df

,id,did_the_patient_expire_in_hospital,hospital_length_of_stay,age,sex,height,weight,smoking_history,previous_er_visit_within_14_days,admission_disposition,...,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99
0,2,No,5,61,0,154.900000,73.600000,0.0,0,1,...,0.001741,-0.000491,0.003457,-0.003979,0.004978,0.004051,-0.000673,-0.004983,-0.003847,0.006019
1,3,No,7,58,0,166.911029,80.265922,0.0,0,1,...,0.001509,-0.001390,0.002812,-0.008017,-0.001268,0.002852,0.006459,-0.002609,-0.009086,0.005165
2,6,No,32,44,1,177.000000,112.000000,0.0,0,0,...,0.001509,-0.001390,0.002812,-0.008017,-0.001268,0.002852,0.006459,-0.002609,-0.009086,0.005165
3,7,No,4,50,1,188.000000,91.100000,0.0,0,1,...,0.009071,0.008938,-0.008208,-0.003012,0.009887,0.005104,-0.001588,-0.008692,0.002962,-0.006676
4,8,No,10,72,1,171.000000,110.000000,1.0,0,0,...,0.001400,0.002151,0.002864,-0.001750,0.003259,0.001476,0.005173,-0.004304,-0.000600,0.000457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411,514,No,4,68,0,164.000000,77.100000,0.0,1,1,...,-0.000256,0.001633,0.005509,-0.004061,0.006309,0.004526,0.001771,-0.003166,0.000249,0.001823
412,515,No,2,56,0,165.000000,72.600000,0.0,0,1,...,0.001973,0.000408,0.004101,0.000060,0.011223,0.005250,-0.007805,-0.007356,0.001392,0.006872
413,516,No,13,74,0,166.911029,55.000000,0.0,1,1,...,0.001973,0.000408,0.004101,0.000060,0.011223,0.005250,-0.007805,-0.007356,0.001392,0.006872
414,519,No,9,74,1,166.911029,81.700000,0.0,1,1,...,0.001741,-0.000491,0.003457,-0.003979,0.004978,0.004051,-0.000673,-0.004983,-0.003847,0.006019
